In [1]:
import findspark
findspark.init()

In [14]:
import pyspark
from pyspark.streaming import StreamingContext
from pyspark.mllib.clustering import StreamingKMeans
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
import uuid
job_uuid = 'job_'+ str(uuid.uuid4())
sc = pyspark.SparkContext('local',job_uuid)

In [15]:
ssc = StreamingContext(sc,10)

In [8]:
class Parser():
    def __init__(self,type='train',delimiter=',',num_elements=5,job_uuid=''):
        self.type = type
        self.delimiter = delimiter
        self.num_elements = num_elements
        self.job_uuid = job_uuid
    
    def parse(self,l):
        try:
            line = l.split(self.delimiter)
            if self.type=='test':
                category = float(line[0])
                feature_vector = Vectors.dense(line[1:])
                return LabeledPoint(category, feature_vector)
            elif self.type=='train':
                category = -1
                feature_vector = Vectors.dense(line)
                return LabeledPoint(category, feature_vector)
            else:
                # log exceptions
                f = open('/errors_events/{0}.txt'.format(self.job_uuid),'a')
                f.write('Unknown type: {0}'.format(self.type))
                f.close()
        except:
            # log errors
            f = open('/error_events/{0}.txt'.format(self.job_uuid),'a')
            f.write('Error parsing line: {0}'.format)
            f.close()  
            

In [18]:
num_features = 2
num_clusters = 3

training_parser = Parser('train',',',num_features+1,job_uuid)
test_parser = Parser('test',',',num_features,job_uuid)

trainingData = ssc.textFileStream("/training_data").\
    map(lambda x: training_parser.parse(x)).map(lambda x: x.features).filter(lambda x: x is not None)
testData = ssc.textFileStream("/test_data").\
    map(lambda x: test_parser.parse(x)).filter(lambda x: x is not None)
streaming_clustering = StreamingKMeans(k=num_clusters, decayFactor=1.0).\
    setRandomCenters(num_features,0,0)
streaming_clustering.trainOn(trainingData)
streaming_clustering.predictOnValues(testData.map(lambda x: (x.label, x.features))).\
    pprint()

Py4JJavaError: An error occurred while calling o69.textFileStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.dstream.InputDStream.<init>(InputDStream.scala:45)
	at org.apache.spark.streaming.dstream.FileInputDStream.<init>(FileInputDStream.scala:82)
	at org.apache.spark.streaming.StreamingContext.fileStream(StreamingContext.scala:359)
	at org.apache.spark.streaming.StreamingContext$$anonfun$textFileStream$1.apply(StreamingContext.scala:415)
	at org.apache.spark.streaming.StreamingContext$$anonfun$textFileStream$1.apply(StreamingContext.scala:415)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.streaming.StreamingContext.withNamedScope(StreamingContext.scala:2

-------------------------------------------
Time: 2018-03-24 13:00:20
-------------------------------------------

-------------------------------------------
Time: 2018-03-24 13:00:30
-------------------------------------------

-------------------------------------------
Time: 2018-03-24 13:00:40
-------------------------------------------

-------------------------------------------
Time: 2018-03-24 13:00:50
-------------------------------------------



In [19]:
ssc.stop()

In [20]:
streaming_clustering.latestModel().clusterCenters

array([[ 1.76405235,  0.40015721],
       [ 0.97873798,  2.2408932 ],
       [ 1.86755799, -0.97727788]])

In [21]:
streaming_clustering.latestModel().centers

array([[ 1.76405235,  0.40015721],
       [ 0.97873798,  2.2408932 ],
       [ 1.86755799, -0.97727788]])